In [2]:
import pandas as pd
from pathlib import Path
import requests
import json

In [3]:
def log_df(df, nrows=5):
    print(df.shape)
    return df.head(nrows)

In [4]:
raw_data_paths = Path('../data').rglob('observations*.csv')
data_paths = [str(path) for path in raw_data_paths]
print(data_paths)

['../data/clarkstown-high-school-north/observations-200303.csv', '../data/cedar-creek-reserve/observations-199064.csv', '../data/los-angeles-bioblitz/observations-190446.csv', '../data/ciencia-ciudadana-peru-bats/observations-199065.csv', '../data/ciencia-ciudadana-peru-bees/observations-199066.csv']


---
# make taxa index species count match inaturalist

In [5]:
file = '../data/clarkstown-high-school-north/observations-200303.csv'
df = pd.read_csv(file, dtype=str)
log_df(df)

(1086, 42)


,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,created_at,updated_at,quality_grade,...,common_name,iconic_taxon_name,taxon_id,taxon_kingdom_name,taxon_phylum_name,taxon_class_name,taxon_order_name,taxon_family_name,taxon_genus_name,taxon_species_name
0,1551812,2015-05-27 08:30:00,2015-05-27,2015-05-27 05:30:00 -0700,Eastern Time (US & Canada),97128,lgertzer,2015-05-27 07:50:23 -0700,2019-05-12 16:17:45 -0700,research,...,Lettered Sphinx,Insecta,143452,Animalia,Arthropoda,Insecta,Lepidoptera,Sphingidae,Deidamia,Deidamia inscriptum
1,1556119,Thu May 28 2015 09:12:41 GMT-0400 (EDT),2015-05-28,2015-05-28 06:12:41 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:24 -0700,2016-08-29 10:44:52 -0700,casual,...,maples,Plantae,47727,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Sapindaceae,Acer,NaN
2,1556120,Thu May 28 2015 09:14:17 GMT-0400 (EDT),2015-05-28,2015-05-28 06:14:17 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:43 -0700,2016-02-16 19:05:32 -0800,research,...,ribwort plantain,Plantae,53178,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,Plantago lanceolata
3,1556121,Thu May 28 2015 09:13:47 GMT-0400 (EDT),2015-05-28,2015-05-28 06:13:47 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2016-11-21 17:49:19 -0800,needs_id,...,Kentucky bluegrass,Plantae,60307,Plantae,Tracheophyta,Liliopsida,Poales,Poaceae,Poa,Poa pratensis
4,1556122,Thu May 28 2015 09:13:07 GMT-0400 (EDT),2015-05-28,2015-05-28 06:13:07 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2021-03-04 19:19:08 -0800,needs_id,...,dicots,Plantae,47124,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN


In [164]:
df['is_species'] = df['taxon_species_name'].notna()
adjust_is_species_for_higher_ranks(df)

In [165]:
inat_taxa = [
"Taraxacum officinale",
"Trifolium repens",
"Turdus migratorius",
"Rosa multiflora",
"Odocoileus virginianus",
"Sciurus carolinensis",
"Tamias striatus",
"Anas platyrhynchos",
"Branta canadensis",
"Trifolium pratense",
"Marmota monax",
"Glechoma hederacea",
"Plantago lanceolata",
"Artemisia vulgaris",
"Toxicodendron radicans",
"Camponotus pennsylvanicus",
"Harmonia axyridis",
"Parthenocissus quinquefolia",
"Dactylis glomerata",
"Ailanthus altissima",
"Nuphar",
"Poa pratensis",
"Rubus phoenicolasius",
"Cygnus olor",
"Agelaius phoeniceus",
"Sturnus vulgaris",
"Chelydra serpentina",
"Canis familiaris",
"Apis mellifera",
"Phytolacca americana",
"Armadillidium vulgare",
"Capsella bursa-pastoris",
"Rumex obtusifolius",
"Picea",
"Plantago rugelii",
"Formica fusca",
"Cyanocitta cristata",
"Passer domesticus",
"Homo sapiens",
"Theridiidae",
"Fagus grandifolia",
"Oxalis stricta",
"Liriodendron tulipifera",
"Hedera helix",
"Limenitis archippus",
"Verbascum thapsus",
"Acer palmatum",
"Marasmius oreades",
"Tilia cordata",
"Persicaria longiseta",
"Formica pallidefulva",
"Zenaida macroura",
"Ardea herodias",
"Corvus brachyrhynchos",
"Cardinalis cardinalis",
"Melospiza melodia",
"Quiscalus quiscula",
"Dumetella carolinensis",
"Vulpes vulpes",
"Sylvilagus floridanus",
"Hesperis matronalis",
"Iris pseudacorus",
"Ranunculus repens",
"Philoscia muscorum",
"Typha latifolia",
"Stellaria",
"Sisyrinchium angustifolium",
"Poa annua",
"Cyprinus carpio",
"Quercus alba",
"Chelidonium majus",
"Frangula alnus",
"Alliaria petiolata",
"Paeonia",
"Toxicodendron rydbergii",
"Plantago major",
"Rhododendron ponticum",
"Papilio glaucus",
"Limenitis arthemis",
"Xysticus",
"Lavandula",
"Viburnum acerifolium",
"Phragmites australis",
"Celastrus orbiculatus",
"Campaea perlata",
"Hosta",
"Periplaneta americana",
"Rhododendron maximum",
"Felis catus",
"Musca domestica",
"Entodon seductrix",
"Veronica officinalis",
"Platycladus orientalis",
"Ardea alba",
"Ranunculus fascicularis",
"Rhus typhina",
"Petunia × atkinsiana",
"Gallus gallus",
"Meleagris gallopavo",
"Aix sponsa",
"Spizella passerina",
"Piranga rubra",
"Molothrus ater",
"Hirundo rustica",
"Tachycineta bicolor",
"Sialia sialis",
"Hylocichla mustelina",
"Toxostoma rufum",
"Tyrannus tyrannus",
"Thamnophis",
"Chrysemys picta",
"Trachemys scripta",
"Terrapene carolina",
"Gopherus polyphemus",
"Procyon lotor",
"Canis latrans",
"Pardosa",
"Taxus",
"Salicaceae",
"Juniperus",
"Cypripedium acaule",
"Sherardia arvensis",
"Geranium maculatum",
"Acer negundo",
"Diplopoda",
"Acer rubrum",
"Castianeira",
"Scarabaeoidea",
"Thuja",
"Oniscus asellus",
"Vanessa cardui",
"Nepeta cataria",
"Oenothera",
"Prunus americana",
"Iris versicolor",
"Hemigrapsus sanguineus",
"Symplocarpus foetidus",
"Quercus rubra",
"Quercus velutina",
"Membracoidea",
"Kalmia",
"Platanus",
"Anthomyiidae",
"Linaria vulgaris",
"Cerastium",
"Chlorophyta",
"Erigeron",
"Moronidae",
"Veronica chamaedrys",
"Digitaria sanguinalis",
"Urtica dioica",
"Sphecius speciosus",
"Pinus resinosa",
"Acer saccharum",
"Equisetum hyemale",
"Prunella vulgaris",
"Achillea",
"Sarcophagidae",
"Cirsium vulgare",
"Hypochaeris radicata",
"Lactuca serriola",
"Vinca major",
"Cotoneaster pannosus",
"Ribes sanguineum",
"Trametes versicolor",
"Abies",
"Corylus avellana",
"Mentha",
"Schizophyllum commune",
"Acer platanoides",
"Quercus macrocarpa",
"Fraxinus",
"Celtis occidentalis",
"Rumex acetosa",
"Vinca minor",
"Lamium purpureum",
"Rosa canina",
"Leucanthemum",
"Robinia",
"Morus alba",
"Persicaria maculosa",
"Thelypteridaceae",
"Philaenus spumarius",
"Acer campestre",
"Ricinus communis",
"Veronica anagallis-aquatica",
"Callitriche",
"Chenopodium album",
"Trombidium",
"Anatrytone logan",
"Berberis thunbergii",
"Muhlenbergia rigens",
"Melissa officinalis",
"Cirsium arvense",
"Paspalum dilatatum",
"Nephrotoma",
"Smilax",
"Pluteus cervinus",
"Ipomoea pes-caprae",
"Chrysopilus thoracicus",
"Camponotus chromaiodes",
"Phalaris arundinacea",
"Dicentra eximia",
"Triticum aestivum",
"Lithobates catesbeianus",
"Geophilomorpha",
"Solidago canadensis",
"Carex vulpinoidea",
"Dysdera crocata",
"Aphaenogaster",
"Myrmica",
"Candelaria concolor",
"Pelargonium",
"Arctium lappa",
"Arenaria serpyllifolia",
"Bromus inermis",
"Buddleja davidii",
"Galium mollugo",
"Gypsophila elegans",
"Lepidium virginicum",
"Lonicera japonica",
"Phyllostachys aurea",
"Polygonum aviculare",
"Ranunculus bulbosus",
"Lumbricus terrestris",
"Potentilla canadensis",
"Platycryptus undatus",
"Halyomorpha halys",
"Rubus allegheniensis",
"Viola sororia",
"Hydrangea",
"Sambucus canadensis",
"Oxalis dillenii",
"Neoscona crucifera",
"Enallagma geminatum",
"Fundulus diaphanus",
"Ischnura posita",
"Rudbeckia laciniata",
"Euonymus alatus",
"Physcia",
"Porodaedalea pini",
"Calliphora vicina",
"Stropharia rugosoannulata",
"Meconema thalassinum",
"Quercus stellata",
"Vespula maculifrons",
"Sporobolus heterolepis",
"Condylostylus",
"Cupido comyntas",
"Plagiomnium",
"Helianthus tuberosus",
"Narcissus papyraceus",
"Ellychnia",
"Ranunculus hispidus",
"Stomolophus meleagris",
"Plutella",
"Securigera varia",
"Carex grayi",
"Bombus vagans",
"Asarum europaeum",
"Zelus luridus",
"Iresine",
"Menispermum",
"Coenosia",
"Chamaecyparis obtusa",
"Buxus sempervirens",
"Philonotis fontana",
"Deidamia inscriptum",
"Menidia menidia",
"Flavoparmelia baltimorensis",
"Sergiolus capulatus",
"Blatta orientalis",
"Sandalus niger",
"Alticini",
"Brassica carinata",
"Carex pedunculata",
"Clematis campaniflora",
"Dianthus chinensis",
"Hieracium murorum",
"Jungermannia exsertifolia",
"Miscanthus sinensis",
"Salvia nemorosa",
"Scolochloa festucacea",
"Viola × wittrockiana",
"Syrphinae",
"Cerrena unicolor",
"Rhododendron indicum",
"Equus caballus",
"Armadillidium nasatum",
"Celiptera frustulum",
"Podabrus tricostatus",
"Tabanus americanus",
"Potentilla indica",
"Avenella flexuosa",
"Rhagio mystaceus",
"Pluteus petasatus",
"Pseudeuophrys erratica",
"Myrmentoma",
"Festuca longifolia",
"Chrysoclista linneella",
"Coryphoideae",
"Iris × germanica",
"Pilosella caespitosa",
"Lysimachia arvensis",
"Dryobates pubescens",
"Lasius",
"Syringa komarowii",
"Schizomantodea",
"Candolleomyces candolleanus"
]

In [166]:
set(inat_taxa) - set(df[df['is_species']]['scientific_name'].unique())
# Iris × germanica
# Membracoidea
# Myrmentoma
# Scarabaeoidea
# Viola × wittrockiana

{'Chrysemys picta',
 'Gallus gallus',
 'Jungermannia exsertifolia',
 'Limenitis arthemis',
 'Membracoidea',
 'Myrmentoma',
 'Polygonum aviculare',
 'Rumex acetosa',
 'Scarabaeoidea',
 'Terrapene carolina'}

In [167]:
set(df[df['is_species']]['scientific_name'].unique()) - set(inat_taxa)


{'Anas platyrhynchos domesticus',
 'Chrysemys picta picta',
 'Formica',
 'Gallus gallus domesticus',
 'Jungermannia exsertifolia cordifolia',
 'Limenitis arthemis astyanax',
 'Polygonum aviculare aviculare',
 'Rumex acetosa acetosa',
 'Terrapene carolina carolina'}

---
# read projects.json

In [171]:
path = Path('../inat_data_explorer/src/lib/data/data.json')
projects_data = pd.read_json(path)
projects_data[0:1]

,username,card_title,card_summary,image,projects
0,mbarton,Los Angeles Bioblitz Challenge,Tempor sint irure incididunt sit voluptate lab...,https://picsum.photos/id/244,"[{'id': 1, 'title': 'Los Angeles Bioblitz Chal..."


In [172]:
files_project = {}

for projects in projects_data['projects']:
    for project in projects:
        for link in project['links']:
            if  'file' not in link:
                continue
            files_project[link['file']] = project['slug']

print(files_project)


{}


---
# fix time

Safari has invalid date error when using `new Date()` with the default iNat date format. Change to standard date format.

In [45]:
file = '../data/clarkstown-high-school-north/observations-200303.csv'
df = pd.read_csv(file )
log_df(df)

(1086, 42)


,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,created_at,updated_at,quality_grade,...,common_name,iconic_taxon_name,taxon_id,taxon_kingdom_name,taxon_phylum_name,taxon_class_name,taxon_order_name,taxon_family_name,taxon_genus_name,taxon_species_name
0,1551812,2015-05-27 08:30:00,2015-05-27,2015-05-27 05:30:00 -0700,Eastern Time (US & Canada),97128,lgertzer,2015-05-27 07:50:23 -0700,2019-05-12 16:17:45 -0700,research,...,Lettered Sphinx,Insecta,143452.0,Animalia,Arthropoda,Insecta,Lepidoptera,Sphingidae,Deidamia,Deidamia inscriptum
1,1556119,Thu May 28 2015 09:12:41 GMT-0400 (EDT),2015-05-28,2015-05-28 06:12:41 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:24 -0700,2016-08-29 10:44:52 -0700,casual,...,maples,Plantae,47727.0,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Sapindaceae,Acer,NaN
2,1556120,Thu May 28 2015 09:14:17 GMT-0400 (EDT),2015-05-28,2015-05-28 06:14:17 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:43 -0700,2016-02-16 19:05:32 -0800,research,...,ribwort plantain,Plantae,53178.0,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,Plantago lanceolata
3,1556121,Thu May 28 2015 09:13:47 GMT-0400 (EDT),2015-05-28,2015-05-28 06:13:47 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2016-11-21 17:49:19 -0800,needs_id,...,Kentucky bluegrass,Plantae,60307.0,Plantae,Tracheophyta,Liliopsida,Poales,Poaceae,Poa,Poa pratensis
4,1556122,Thu May 28 2015 09:13:07 GMT-0400 (EDT),2015-05-28,2015-05-28 06:13:07 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2021-03-04 19:19:08 -0800,needs_id,...,dicots,Plantae,47124.0,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN


In [33]:
df['id']

0        1551812
1        1556119
2        1556120
3        1556121
4        1556122
          ...   
1081    83431861
1082    83431862
1083    83431865
1084    83431868
1085    83683271
Name: id, Length: 1086, dtype: int64

In [38]:
df['time_observed_at'] = df['time_observed_at'].replace('([\d-]+) ([\d:]+) (-\d{4})', r'\1T\2Z', regex = True)
df['time_observed_at'] = df['time_observed_at'].astype(str)

In [39]:
df['time_observed_at']

0       2015-05-27T05:30:00Z
1       2015-05-28T06:12:41Z
2       2015-05-28T06:14:17Z
3       2015-05-28T06:13:47Z
4       2015-05-28T06:13:07Z
                ...         
1081    2021-06-04T13:46:00Z
1082    2021-06-04T13:46:00Z
1083    2021-06-04T13:46:00Z
1084    2021-06-04T13:46:00Z
1085    2021-06-04T07:12:48Z
Name: time_observed_at, Length: 1086, dtype: object

In [40]:
df

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,created_at,updated_at,quality_grade,...,common_name,iconic_taxon_name,taxon_id,taxon_kingdom_name,taxon_phylum_name,taxon_class_name,taxon_order_name,taxon_family_name,taxon_genus_name,taxon_species_name
0,1551812,2015-05-27 08:30:00,2015-05-27,2015-05-27T05:30:00Z,Eastern Time (US & Canada),97128,lgertzer,2015-05-27 07:50:23 -0700,2019-05-12 16:17:45 -0700,research,...,Lettered Sphinx,Insecta,143452.0,Animalia,Arthropoda,Insecta,Lepidoptera,Sphingidae,Deidamia,Deidamia inscriptum
1,1556119,Thu May 28 2015 09:12:41 GMT-0400 (EDT),2015-05-28,2015-05-28T06:12:41Z,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:24 -0700,2016-08-29 10:44:52 -0700,casual,...,maples,Plantae,47727.0,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Sapindaceae,Acer,NaN
2,1556120,Thu May 28 2015 09:14:17 GMT-0400 (EDT),2015-05-28,2015-05-28T06:14:17Z,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:43 -0700,2016-02-16 19:05:32 -0800,research,...,ribwort plantain,Plantae,53178.0,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,Plantago lanceolata
3,1556121,Thu May 28 2015 09:13:47 GMT-0400 (EDT),2015-05-28,2015-05-28T06:13:47Z,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2016-11-21 17:49:19 -0800,needs_id,...,Kentucky bluegrass,Plantae,60307.0,Plantae,Tracheophyta,Liliopsida,Poales,Poaceae,Poa,Poa pratensis
4,1556122,Thu May 28 2015 09:13:07 GMT-0400 (EDT),2015-05-28,2015-05-28T06:13:07Z,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2021-03-04 19:19:08 -0800,needs_id,...,dicots,Plantae,47124.0,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,83431861,2021/06/04 4:46 PM EDT,2021-06-04,2021-06-04T13:46:00Z,Eastern Time (US & Canada),4586286,maxkahann,2021-06-17 13:54:27 -0700,2021-06-17 13:54:27 -0700,needs_id,...,Red-flowering Currant,Plantae,53435.0,Plantae,Tracheophyta,Magnoliopsida,Saxifragales,Grossulariaceae,Ribes,Ribes sanguineum
1082,83431862,2021/06/04 4:46 PM EDT,2021-06-04,2021-06-04T13:46:00Z,Eastern Time (US & Canada),4586286,maxkahann,2021-06-17 13:54:27 -0700,2021-10-13 19:21:37 -0700,research,...,ribwort plantain,Plantae,53178.0,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,Plantago lanceolata
1083,83431865,2021/06/04 4:46 PM EDT,2021-06-04,2021-06-04T13:46:00Z,Eastern Time (US & Canada),4586286,maxkahann,2021-06-17 13:54:28 -0700,2021-08-28 05:25:10 -0700,needs_id,...,dicots,Plantae,47124.0,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN
1084,83431868,2021/06/04 4:46 PM EDT,2021-06-04,2021-06-04T13:46:00Z,Eastern Time (US & Canada),4586286,maxkahann,2021-06-17 13:54:28 -0700,2021-07-12 10:11:15 -0700,needs_id,...,dicots,Plantae,47124.0,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN


In [42]:
df.to_json('temp/foo.json')